# Metatrader 5
___

### Initialize and Login

In [57]:
# %pip install MetaTrader5
# %pip install pandas
# %pip install plotly
# %pip install python-dotenv


In [58]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
from datetime import datetime
import plotly.express as px

# To access .env file
from dotenv import load_dotenv

import os
import time

In [59]:
# Access .env file
load_dotenv(".env")

# Access variables directly
username = int(os.getenv("USER_NAME"))
password = os.getenv("PASSWORD")
server = os.getenv("SERVER")

symbols = os.getenv("SYMBOLS")

In [60]:
# Initialize and log in to MetaTrader 5
# connect to MetaTrader 5
try:
    # Initialize MetaTrader 5
    if mt5.initialize():
        print("MetaTrader successfully initialized")
        
        # Log in to MetaTrader 5
        if mt5.login(login=username, password=password, server=server):
            print("MetaTrader login successful")
                
        else:
            print("MetaTrader login failed")
            mt5.shutdown()  # Ensure shutdown if login fails
    else:
        print("Failed to initialize MetaTrader 5")
except Exception as e:
    print(f"An error occurred: {e}")

MetaTrader successfully initialized
MetaTrader login failed


### Get Data

In [50]:
# Get names of all currency pairs
names= pd.DataFrame(mt5.symbols_get())[93]
names

KeyError: 93

In [ ]:
# Filter for all symbols containing "EUR"
names[names.str.contains("EUR", na=False)]

136    EURAUDm
137    EURCADm
138    EURCHFm
139    EURDKKm
140    EURGBPm
141    EURHKDm
142    EURJPYm
143    EURMXNm
144    EURNOKm
145    EURNZDm
146    EURPLNm
147    EURSEKm
148    EURSGDm
149    EURTRYm
150    EURUSDm
151    EURZARm
210    XAGEURm
214    XAUEURm
Name: 93, dtype: object

In [ ]:
# request ticks from EURUSD
# Retrieve currency data in 1 minute timeframe with a time range
rates =  mt5.copy_rates_range(symbols, mt5.TIMEFRAME_M1, datetime(2024,10,24), datetime.now())
rates

array([(1729728000, 1.07799, 1.0781 , 1.07795, 1.07806, 44, 9, 0),
       (1729728060, 1.07806, 1.07814, 1.07802, 1.07812, 34, 9, 0),
       (1729728120, 1.07812, 1.07816, 1.0781 , 1.07815, 26, 9, 0), ...,
       (1730313780, 1.08684, 1.08688, 1.08681, 1.08682, 15, 9, 0),
       (1730313840, 1.08682, 1.08687, 1.0868 , 1.08687, 10, 9, 0),
       (1730313900, 1.08688, 1.08688, 1.08686, 1.08688,  6, 9, 0)],
      dtype=[('time', '<i8'), ('open', '<f8'), ('high', '<f8'), ('low', '<f8'), ('close', '<f8'), ('tick_volume', '<u8'), ('spread', '<i4'), ('real_volume', '<u8')])

In [ ]:
# Put data in a dataframe
eur_usd = pd.DataFrame(rates)

# Correct time columns into datetime format
eur_usd['time'] = pd.to_datetime(eur_usd['time'], unit='s')

# Assess ending of data
eur_usd.tail()


,time,open,high,low,close,tick_volume,spread,real_volume
6872,2024-10-30 18:41:00,1.08677,1.08688,1.08677,1.08686,15,9,0
6873,2024-10-30 18:42:00,1.08687,1.08688,1.08683,1.08684,21,9,0
6874,2024-10-30 18:43:00,1.08684,1.08688,1.08681,1.08682,15,9,0
6875,2024-10-30 18:44:00,1.08682,1.08687,1.08680,1.08687,10,9,0
6876,2024-10-30 18:45:00,1.08688,1.08688,1.08686,1.08688,6,9,0


In [ ]:
# Accessing Account info
account_info =  mt5.account_info()._asdict()
account_info

{'login': 191406440,
 'trade_mode': 0,
 'leverage': 2000,
 'limit_orders': 1024,
 'margin_so_mode': 0,
 'trade_allowed': True,
 'trade_expert': True,
 'margin_mode': 2,
 'currency_digits': 2,
 'fifo_close': False,
 'balance': 91.8,
 'credit': 0.0,
 'profit': 0.0,
 'equity': 91.8,
 'margin': 0.0,
 'margin_free': 91.8,
 'margin_level': 0.0,
 'margin_so_call': 60.0,
 'margin_so_so': 0.0,
 'margin_initial': 0.0,
 'margin_maintenance': 0.0,
 'assets': 0.0,
 'liabilities': 0.0,
 'commission_blocked': 0.0,
 'name': 'kay_demo',
 'server': 'Exness-MT5Trial',
 'currency': 'USD',
 'company': 'Exness Technologies Ltd'}

In [ ]:
# Total symbols available
total_symbols = mt5.symbols_total()

print("Total symbols available:", total_symbols)

Total symbols available: 395


In [ ]:
# # Chart
# fig = px.line(eur_usd, x=eur_usd['time'], y=eur_usd['close'], title='EUR/USD Chart')
# fig.show()

### Make Trade

In [10]:
# Initialize trading parameters
symbols = "EURUSDm"
lot = 0.1
price_buy = mt5.symbol_info_tick(symbols).ask
point = mt5.symbol_info(symbols).point
deviation = 20

# Set up the trading request
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbols,
    "volume": lot,
    "type": mt5.ORDER_TYPE_BUY, # buy order
    #"type": mt5.ORDER_TYPE_SELL, # sell order
    "price": mt5.symbol_info_tick(symbols).ask,
    # "sl": price_buy - (20 * point),
    # "tp": price_buy + (20 * point),
    "comment": "python script open",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,
}

# Send the trading request
result = mt5.order_send(request)

# Check and print the execution result with detailed diagnostics
print("Order: by {} {} lots at {} ".format(symbols, lot, price_buy))
if result.retcode == mt5.TRADE_RETCODE_DONE:
    print("Order successful")
elif result.retcode == 10004:
    print("Order failed with error code:", result.retcode)
    print("TRADE_RETCODE_REQUOTE - Price has changed; a new quote is required.")
elif result.retcode == 10008:
    print("Order succeeded with code:", result.retcode)
    print("TRADE_RETCODE_PLACED - The order has been placed successfully.")
elif result.retcode == 10013:
    print("Order failed with error code:", result.retcode)
    print("TRADE_RETCODE_INVALID - Invalid trade parameters.")
elif result.retcode == 10014:
    print("Order failed with error code:", result.retcode)
    print("TRADE_RETCODE_INVALID_VOLUME - The volume specified is incorrect.")
elif result.retcode == 10016:
    print("Order failed with error code:", result.retcode)
    print("TRADE_RETCODE_INVALID_STOPS - The stop loss or take profit is invalid.")
elif result.retcode == 10017:
    print("Order failed with error code:", result.retcode)
    print("TRADE_RETCODE_TRADE_DISABLED - Trading is disabled on this account.")
elif result.retcode == 10018:
    print("Order failed with error code:", result.retcode)
    print("TRADE_RETCODE_MARKET_CLOSED - The market is currently closed.")
elif result.retcode == 10019:
    print("Order failed with error code:", result.retcode)
    print("TRADE_RETCODE_NO_MONEY - Insufficient funds to execute the trade.")
elif result.retcode == 10026:
    print("Order failed with error code:", result.retcode)
    print("TRADE_RETCODE_SERVER_DISABLES_AT - The server disabled trading at this time.")
elif result.retcode == 10027:
    print("Order failed with error code:", result.retcode)
    print("TRADE_RETCODE_CLIENT_DISABLES_AT - The client disabled trading at this time.")
elif result.retcode == 10034:
    print("Order failed with error code:", result.retcode)
    print("TRADE_RETCODE_LIMIT_VOLUME - The requested volume exceeds the limit.")
elif result.retcode == 10011:
    print("Order failed with error code:", result.retcode)
    print("TRADE_RETCODE_ERROR - General trading error.")
elif result.retcode == 10012:
    print("Order failed with error code:", result.retcode)
    print("TRADE_RETCODE_TIMEOUT - The trade request timed out.")
elif result.retcode == 10015:
    print("Order failed with error code:", result.retcode)
    print("TRADE_RETCODE_INVALID_PRICE - The specified price is invalid.")
elif result.retcode == 10020:
    print("Order failed with error code:", result.retcode)
    print("TRADE_RETCODE_PRICE_CHANGED - The price has changed; resubmit the request.")
elif result.retcode == 10021:
    print("Order failed with error code:", result.retcode)
    print("TRADE_RETCODE_PRICE_OFF - The specified price is currently unavailable.")
elif result.retcode == 10024:
    print("Order failed with error code:", result.retcode)
    print("TRADE_RETCODE_TOO_MANY_REQUESTS - Too many requests sent in a short time.")
elif result.retcode == 10031:
    print("Order failed with error code:", result.retcode)
    print("TRADE_RETCODE_CONNECTION - Lost connection to the server.")
elif result.retcode == 0:
    print("Order successful with code:", result.retcode)
    print("ERR_SUCCESS - The request was successful.")
elif result.retcode == 4752:
    print("Order failed with error code:", result.retcode)
    print("ERR_TRADE_DISABLED - Trading is disabled.")
elif result.retcode == 4753:
    print("Order failed with error code:", result.retcode)
    print("ERR_TRADE_POSITION_NOT_FOUND - The specified position could not be found.")
elif result.retcode == 4754:
    print("Order failed with error code:", result.retcode)
    print("ERR_TRADE_ORDER_NOT_FOUND - The specified order could not be found.")
elif result.retcode == 4755:
    print("Order failed with error code:", result.retcode)
    print("ERR_TRADE_DEAL_NOT_FOUND - The specified deal could not be found.")
else:
    print("Order failed with unknown error code:", result.retcode)
    print("Error details:", result)


Order: by EURUSDm 0.1 lots at 1.07112 
Order successful


### Cancel Previous Trade

In [6]:
# Get Order ticket
mt5.positions_get()[0]._asdict()['ticket']

1063640333

In [7]:
# Get the order number of previous trade
order_number = mt5.positions_get()[0]._asdict()['ticket']

# Closing the buy order placed earlier with a sell order and its order number
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbols,
    "volume": lot,
    "type": mt5.ORDER_TYPE_SELL, # sell order
    "position": order_number,
    "comment": "close open order",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,
}

close = mt5.order_send(request)
if result.retcode == mt5.TRADE_RETCODE_DONE:
    print("Order successfully closed")
else:
    print("Order failed with error code:", close.retcode)

Order successfully closed


## Trading Bot

### Lots, sl & tp Preparations

#### Retcode

In [9]:
# Mapping retcodes/errors to messages
retcode_messages = {
    10004: "Price has changed; a new quote is required.",
    10013: "Invalid trade parameters.",
    10014: "The volume specified is incorrect.",
    10016: "The stop loss or take profit is invalid.",
    10017: "Trading is disabled on this account.",
    10018: "The market is currently closed.",
    10019: "Insufficient funds to execute the trade.",
    10026: "The server disabled trading at this time.",
    10027: "The client disabled trading at this time.",
    10034: "The requested volume exceeds the limit.",
    10011: "General trading error.",
    10012: "The trade request timed out.",
    10015: "The specified price is invalid.",
    10020: "The price has changed; resubmit the request.",
    10021: "The specified price is currently unavailable.",
    10024: "Too many requests sent in a short time.",
    10031: "Lost connection to the server.",
    0: "ERR_SUCCESS - The request was successful.",
    4752: "Trading is disabled.",
    4753: "The specified position could not be found.",
    4754: "The specified order could not be found.",
    4755: "The specified deal could not be found.",
}

#### EURUSDm

In [51]:
# Trading Bot
symbol = 'XAUUSDm'
lot = 0.01
point = mt5.symbol_info(symbol).point 
tp_amount = 1
sl_amount = 1


buy_order_type = mt5.ORDER_TYPE_BUY
sell_order_type = mt5.ORDER_TYPE_SELL

# Set up the trading request
buy_price = mt5.symbol_info_tick(symbol).ask
sell_price = mt5.symbol_info_tick(symbol).bid

# Compute pip values
tp_pips = tp_amount * point * 100
sl_pips = sl_amount * point * 100

# Stop loss and take profit using pips 60pips
buy_sl = buy_price - (sl_pips) 
buy_tp = buy_price + (tp_pips)

sell_sl = sell_price + (sl_pips)
sell_tp = sell_price - (tp_pips)


def create_order(symbol, lot, order_type, price, sl, tp):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": order_type,
        "price": price,
        "sl": sl,
        "tp": tp,
        "comment": "python open",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    order = mt5.order_send(request)
    
    # Handle the response code
    if order.retcode in [10009, 10008]:
        pass  # Continue with next code
    else:
        # Retrieve the message if retcode is known; else print full order details for unknown retcode
        message = retcode_messages.get(order.retcode)
        if message:
            print(message)
        else:
            print(f"Order failed with unknown error code: {order.retcode}")
            print("Error details:", order)


AttributeError: 'NoneType' object has no attribute 'point'

In [ ]:
create_order(symbol, lot, buy_order_type, buy_price, buy_sl, sell_sl)

#### XAUUSD

- At 1 lot size at 100pips = $100
- At 0.1 lot size at 100pips = $10
- At 0.01 lot size at 100pips = $1

In [ ]:
mt5.symbol_info('XAUUSDm')

In [11]:

# Trading Bot
symbol = 'XAUUSDm'
lot = 0.01
point = mt5.symbol_info(symbol).point 
tp_amount = 10
sl_amount = 10


buy_order_type = mt5.ORDER_TYPE_BUY
sell_order_type = mt5.ORDER_TYPE_SELL

# Set up the trading request
buy_price = mt5.symbol_info_tick(symbol).ask
sell_price = mt5.symbol_info_tick(symbol).bid

# Compute pip values
tp_pips = tp_amount * point * 1000
sl_pips = sl_amount * point * 1000

# Stop loss and take profit using pips 60pips
buy_sl = buy_price - (sl_pips) 
buy_tp = buy_price + (tp_pips)

sell_sl = sell_price + (sl_pips)
sell_tp = sell_price - (tp_pips)


def create_order(symbol, lot, order_type, price, sl, tp):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": order_type,
        "price": price,
        "sl": sl,
        "tp": tp,
        "comment": "python open",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    order = mt5.order_send(request)
    
    # Handle the response code
    if order.retcode in [10009, 10008]:
        pass  # Continue with next code
    else:
        # Retrieve the message if retcode is known; else print full order details for unknown retcode
        message = retcode_messages.get(order.retcode)
        if message:
            print(message)
        else:
            print(f"Order failed with unknown error code: {order.retcode}")
            print("Error details:", order)
            
create_order(symbol, lot, buy_order_type, buy_price, buy_sl, sell_sl)


AttributeError: 'NoneType' object has no attribute 'point'

#### BTCUSDm

In [274]:
# Trading Bot
symbol = 'BTCUSDm'
lot = 0.01
symbol_info = mt5.symbol_info(symbol)
point = mt5.symbol_info(symbol).point 
spread = symbol_info.spread * point

buy_order_type = mt5.ORDER_TYPE_BUY
sell_order_type = mt5.ORDER_TYPE_SELL

# Set up the trading request
buy_price = mt5.symbol_info_tick(symbol).ask
sell_price = mt5.symbol_info_tick(symbol).bid

# Stop loss and take profit using pips 450pips (0.01) lots pips = 0.50$, 450 pips (0.02 lots) = 1.08
buy_sl = (buy_price) - (5 * 900 * point * 10) 
buy_tp = (buy_price) + (5 * 900 * point * 10)

sell_sl = (sell_price) + (5 * 900 * point * 10)
sell_tp = (sell_price) - (5 * 900 * point * 10)


def create_order(symbol, lot, order_type, price, sl, tp):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": order_type,
        "price": price,
        "sl": sl,
        "tp": tp,
        "comment": "python open",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    order = mt5.order_send(request)
    
    # Handle the response code
    if order.retcode in [10009, 10008]:
        pass  # Continue with next code
    else:
        # Retrieve the message if retcode is known; else print full order details for unknown retcode
        message = retcode_messages.get(order.retcode)
        if message:
            print(message)
        else:
            print(f"Order failed with unknown error code: {order.retcode}")
            print("Error details:", order)


In [275]:
create_order(symbol, lot, buy_order_type, buy_price, buy_sl, buy_tp)

In [183]:
buy_price

103028.16

In [184]:
spread

28.8

In [186]:
print(buy_tp, buy_sl)

103066.96 103018.16


In [123]:
symbol_info.volume_step

0.01

In [161]:
def calculate_lot_size(symbol, risk_amount, stop_loss_pips):
    """
    Calculates the lot size based on the fixed risk amount and stop loss distance.
    """
    # Get symbol info
    symbol_info = mt5.symbol_info(symbol)
    if not symbol_info:
        print(f"Symbol {symbol} not found.")
        return None

    # Tick value and point
    tick_value = symbol_info.trade_tick_value  # Value of one tick for one lot
    point = symbol_info.point  # The minimum price change (e.g., 0.01 for BTCUSDm)

    # Stop loss in price terms
    stop_loss_price = stop_loss_pips * point  # Stop loss in price terms

    # Lot size formula
    lot_size = risk_amount / (stop_loss_price * tick_value)

    # Ensure lot size adheres to broker constraints
    lot_size = max(symbol_info.volume_min, min(lot_size, symbol_info.volume_max))
    lot_size = round(lot_size / symbol_info.volume_step) * symbol_info.volume_step

    return lot_size


# Example usage
symbol = "BTCUSDm"  # Trading symbol
risk_amount = 4  # Risk per trade in USD
stop_loss_pips = 10  # Stop loss distance in pips

lot_size = calculate_lot_size(symbol, risk_amount, stop_loss_pips)
if lot_size:
    print(f"Calculated lot size for {symbol}: {lot_size}")
else:
    print("Failed to calculate lot size.")



Symbol BTCUSDm not found.
Failed to calculate lot size.


In [157]:
def calculate_lot_size(symbol, stop_loss_pips, risk_amount):
    """
    Calculate the appropriate lot size for a given symbol based on fixed risk amount and stop loss in pips.
    
    Args:
        symbol (str): The trading symbol (e.g., "EURUSD")
        stop_loss_pips (float): Stop loss distance in pips
        risk_amount (float): Fixed risk amount in dollars (default: $1)
    
    Returns:
        float: Calculated lot size respecting broker constraints
    """
    # Get symbol information
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        raise ValueError(f"Symbol {symbol} not found")
    
    # Extract symbol properties
    pip_value = symbol_info.point
    digits = symbol_info.digits
    contract_size = symbol_info.trade_contract_size
    volume_min = symbol_info.volume_min
    volume_max = symbol_info.volume_max
    volume_step = symbol_info.volume_step
    
    # Convert pip value based on digits
    if digits == 3 or digits == 5:
        pip_value *= 10
    
    # Calculate pip value in account currency (USD)
    # For simplicity, assuming USD account. For other base currencies, 
    # you would need to apply the conversion rate
    pip_value_usd = (pip_value * contract_size)
    
    # Calculate required lot size for $1 risk
    lot_size = risk_amount / (stop_loss_pips * pip_value_usd)
    
    # Round to nearest volume step
    lot_size = round(lot_size / volume_step) * volume_step
    
    # Ensure lot size is within broker constraints
    lot_size = max(volume_min, min(volume_max, lot_size))
    
    return lot_size

def validate_lot_size(symbol, lot_size, stop_loss_pips):
    """
    Validate the calculated lot size and return actual risk amount.
    
    Args:
        symbol (str): The trading symbol
        lot_size (float): Calculated lot size
        stop_loss_pips (float): Stop loss distance in pips
    
    Returns:
        dict: Validation results including actual risk amount
    """
    symbol_info = mt5.symbol_info(symbol)
    pip_value = symbol_info.point
    digits = symbol_info.digits
    contract_size = symbol_info.trade_contract_size
    
    if digits == 3 or digits == 5:
        pip_value *= 10
        
    actual_risk = lot_size * stop_loss_pips * pip_value * contract_size
    
    return {
        "lot_size": lot_size,
        "actual_risk_usd": round(actual_risk, 2),
        "stop_loss_pips": stop_loss_pips,
        "within_constraints": lot_size >= symbol_info.volume_min and lot_size <= symbol_info.volume_max
    }

In [ ]:
def close_order(symbol, lot, order_type, position):
    request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    'position': position,
    "type": order_type, # sell order
    "comment": "closed order",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,
    }
    order = mt5.order_send(request)
    
    # Handle the response code
    if order.retcode in [10009, 10008]:
        pass  # Continue with next code
    else:
        # Retrieve the message if retcode is known; else print full order details for unknown retcode
        message = retcode_messages.get(order.retcode)
        if message:
            print(message)
        else:
            print(f"Order failed with unknown error code: {order.retcode}")
            print("Error details:", order)

### Bot

In [52]:
while True:
    # Retrieve currency data in 1-minute timeframe with a time range
    rates = pd.DataFrame(mt5.copy_rates_range(symbols, mt5.TIMEFRAME_M3, datetime(2024, 10, 24), datetime.now()))
    rates['time'] = pd.to_datetime(rates['time'], unit='s')

    # Define OHLC values for current and previous candles
    current_close = rates.iloc[-1]['close']
    last_close = rates.iloc[-2]['close']
    last_high = rates.iloc[-2]['high']
    last_low = rates.iloc[-2]['low']

    # Define conditions
    long_condition = current_close > last_high
    short_condition = current_close < last_low
    close_long_condition = current_close < last_close
    close_short_condition = current_close > last_close

    # Check for open positions and set flags
    positions = mt5.positions_get()
    no_positions = len(positions) == 0
    already_buy = any(pos._asdict()['type'] == 0 for pos in positions)
    already_sell = any(pos._asdict()['type'] == 1 for pos in positions)

    # Execution logic for placing and closing orders
    if long_condition and no_positions:
        create_order(symbol, lot, buy_order_type, buy_price, buy_sl, buy_tp)
        print('Buy Order Placed')
    elif long_condition and already_sell:
        position = positions[0]._asdict()['ticket']
        close_order(symbol, lot, sell_order_type, position)
        print('Sell Position Closed')
        time.sleep(1)
        create_order(symbol, lot, buy_order_type, buy_price, buy_sl, buy_tp)
    elif long_condition and already_buy:
        print('Active Buy Position')

    if short_condition and no_positions:
        create_order(symbol, lot, sell_order_type, sell_price, sell_sl, sell_tp)
        print('Sell Order Placed')
    elif short_condition and already_buy:
        position = positions[0]._asdict()['ticket']
        close_order(symbol, lot, buy_order_type, position)
        print('Buy Position Closed')
        time.sleep(1)
        create_order(symbol, lot, sell_order_type, sell_price, sell_sl, sell_tp)
    elif short_condition and already_sell:
        print('Active Sell Position')

    # Close all 'buy' positions if `close_long_condition` is met
    if close_long_condition and already_buy:
        for pos in positions:
            if pos._asdict()['type'] == 0:  # 'buy' type
                ticket = pos._asdict()['ticket']
                close_order(symbol, lot, buy_order_type, ticket)
                print("Buy Position Closed")
    # Close all 'sell' positions if `close_short_condition` is met
    if close_short_condition and already_sell:
        for pos in positions:
            if pos._asdict()['type'] == 1:  # 'sell' type
                ticket = pos._asdict()['ticket']
                close_order(symbol, lot, sell_order_type, ticket)
                print("Sell Position Closed")

    # Wait for 3 minute before the next iteration
    time.sleep(180)


KeyError: 'time'